# Импорты

In [4]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import stats

# Преобразовываем датасет

In [13]:
def vectorize_text(df):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['text'])
    y = df['class'].values
    return X, y, vectorizer.get_feature_names_out(), vectorizer

df = pd.read_csv('data/SMS.tsv', sep='\t')
X, y, feature_names, vectorizer = vectorize_text(df)

In [26]:
len(feature_names)

8713

# Фильтрирующий метод отбора признаков

## Моя реализация

In [16]:
def select_k_best_features_ttest(X, y, feature_names, k):
    p_values = []
    # Проходимся по всем признакам
    for index in range(X.shape[1]):
        group1 = X[y == 0][:, index].toarray()
        group2 = X[y == 1][:, index].toarray()
        # Если признак не встречается в одной из групп, пропускаем
        if np.sum(group1) == 0 or np.sum(group2) == 0:
            p_values.append(1)  # Максимальное p-value для пропусков
            continue
        t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=False)
        p_values.append(p_val)

    # Сортируем по p-value и выбираем k лучших
    p_value_df = pd.DataFrame({'feature': feature_names, 'p_value': p_values})
    best_features = p_value_df.nsmallest(k, 'p_value')['feature'].tolist()
    return best_features

In [24]:
# Отбор признаков
k = 30  # Количество признаков для выбора
best_features = select_k_best_features_ttest(X, y, feature_names, k)
print("Наиболее значимые признаки (слова):", best_features)

Наиболее значимые признаки (слова): ['00', '000', '000pes', '008704050406', '0089', '0121', '01223585236', '01223585334', '0125698789', '02', '0207', '02072069400', '02073162414', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07', '07008009200', '07046744435', '07090201529', '07090298926', '07099833605', '07123456789', '0721072']


## SKlearn реализация

In [ ]:
# создаем объект SelectKBest с указанием метода отбора признаков (например, chi2)
selector = SelectKBest(score_func=chi2, k=5) # отбираем 5 лучших признаков

# подготавливаем данные (X - матрица признаков, y - вектор меток)
X_new = selector.fit_transform(X, y)

# получаем индексы отобранных признаков
selected_features = selector.get_support(indices=True)

# выводим индексы отобранных признаков
print(selected_features)